# Building a Decision Tree Algorithm

This notebook will take you through the steps involved in building a decision tree algorithm from scratch. Remember that scikit-learn provides its own implementation of the decision tree algorithm; and most of the time this is what you'll want to use. However, it's always useful to know what's happening under the hood, if you ever need functionality that isn't implemented out of the box, then this knowledge will let you build the algorithm yourself.



## Calculating Entropy

The concept of **entropy** is fundamental to the decision tree algorithm. Entropy is a rough guide to how *mixed up* a dataset is. As a reminder, here's the algorithm for entropy.

![Entropy Formula](entropy_formula.png)

Let's break that down; the partial entropy for each possible *level*, *i* is, (the probability of choosing a label with the value *i*) * (log to the base 2 of the probability of choosing a label with the value of *i*). We calculate the partial entropy for each level, sum them all together and multiply the result by -1.

We'll start by defining a function to calculate the partial entropy for a given level with respect to a categorical feature (the bit to the right of SIGMA, above). Our calculate_partial_entropy function will take a dataframe as a parameter, along with the column name we're calculating entropy for, and the particular value (we'll do this for each possible value of the target column).

### Method

To work out the probability of choosing a value at random we just find the percentage of rows which have that value. Take the census dataset as an example.

In [1]:
from typing import Dict
import pandas as pd
import numpy as np
import math

df = pd.read_csv('vegetation.csv')
df = df.set_index('ID')
df

,Stream,Slope,Elevation,Vegetation
ID,,,,
1,False,steep,high,chapparal
2,True,moderate,low,riparian
3,True,steep,medium,riparian
4,False,steep,medium,chapparal
5,False,flat,high,conifer
6,True,steep,highest,conifer
7,True,steep,high,chapparal


Let's say we want to work out the entropy of the Vegetation column. We'll start by working out the partial entropy of `chapparal`. What's the probability of choosing the value `chapparal` if we choose a row at random from this dataset?

The probability of choosing a particular value *v* from a dataset *d* is 

<pre>
    number of rows with value <em>v</em> / number of rows in <em>d</em>
</pre>

Pandas lets us count the number of rows in a dataframe using the `len()` function. To find the number of rows with value *v* we first filter the dataframe and then count the rows. Applying this, we can calculate the probability of choosing `chapparal` from the dataset above as


## Pandas Recap

The Pandas library gives us a python equivalent of R's dataframes. We can access a particular column in a pandas dataframe using sqaure-bracket notation. To return only the vegetation column in our pandas dataframe we can use


In [2]:
df['Vegetation']

ID
1    chapparal
2     riparian
3     riparian
4    chapparal
5      conifer
6      conifer
7    chapparal
Name: Vegetation, dtype: object

We can filter and subset a pandas dataframe using the `.loc` property (short for *locate*). `.loc` allows us to use logical indexing, returning only rows that meet a specific criterion, for example, to return only rows with a value of `medium` for elevation we can use the following

In [3]:
df.loc[df['Elevation'] == 'medium']

,Stream,Slope,Elevation,Vegetation
ID,,,,
3,True,steep,medium,riparian
4,False,steep,medium,chapparal


The `len()` function returns the total number of rows in a pandas dataframe.

In [4]:
len(df)

7

We're going to find ourselves needing to find the *probability of selecting a value at random* repeatedly when building decision trees. Essentially this is the percentage of rows containing a given value. Put the above information together to create a function which calculates this for us.

In [5]:
def calculate_probability(df: pd.DataFrame, column: str, val: any):
    return len(df.loc[df[column] == val]) / len(df)

calculate_probability(df, 'Elevation', 'medium') # 0.286...
calculate_probability(df, 'Slope', 'steep') # 0.714...

0.7142857142857143

In [7]:
import pandas as pd
import math

# Copy the implementation of this function from the cell above
# It will come in handy when calculating partial entropy
def calculate_probability(df: pd.DataFrame, column: str, val: any):
    return len(df.loc[df[column] == val]) / len(df)

# partial_entropy is defined as the probability of selecting the target level at random
# multiply by log to the base 2 of that probability
# we'll multiply the result by -1 to ensure a positive result
def calculate_partial_entropy(df: pd.DataFrame, column_name: str, target_level: str):
    prob = calculate_probability(df, column_name, target_level)
    return prob * math.log2(prob) * -1

p_ent_chapparal = calculate_partial_entropy(df, 'Vegetation', 'chapparal')
p_ent_riparian = calculate_partial_entropy(df, 'Vegetation', 'riparian')
p_ent_conifer = calculate_partial_entropy(df, 'Vegetation', 'conifer')

print(f"The partial entropy of `chapparal` is {p_ent_chapparal}") # -0.524...
print(f"The partial entropy of `riparian` is {p_ent_riparian}") # -0.516...
print(f"The partial entropy of `conifer` is {p_ent_conifer}") # -0.516...

The partial entropy of `chapparal` is 0.5238824662870492
The partial entropy of `riparian` is 0.5163871205878868
The partial entropy of `conifer` is 0.5163871205878868


### Calculating Total Entropy

We've worked out how to calculate the *partial entropy*, in order to calculate the total entropy of a dataset for a column, we need to add together the partial entropy for each possible value and multiply the result by -1.

The `unique()` method of a pandas column returns a list of each of the unique values found in that column (unsurprisingly). We can now put together the full algorithm for calculating the entropy of a given dataset

1. Get all unique label values for a column
2. For each unique value, calculate the partial entropy
3. Sum all partial entropies
4. Multiply the result by -1

In [8]:
def calculate_entropy(df: pd.DataFrame, column_name: str):
    def calculate_partial_entropy(target_level: str):
        prob = len(df.loc[df[column_name] == target_level]) / len(df)
        partial_entropy = prob * math.log2(prob) * -1
        return partial_entropy
    
    return sum([calculate_partial_entropy(v) for v in df[column_name].unique()])

calculate_entropy(df, "Vegetation") # 1.557

1.5566567074628228

## Calculating Information Gain

Now that we can calculate the entropy of any given dataset, we can work out how much information would be gained by splitting a dataset on a given column. Information Gain is, roughly speaking, the amount by which we expect to reduce entropy after splitting a dataset on a given column. The formula for information gain is given by

![Information Gain Formula](information_gain.png)


In the equation above, the |vertical bars| refer to the **cardinality** or number of rows. |S<sub>v</sub>| is the number of rows containing the value *v*, |S| is the total number of rows. This is essentially the same thing as the probability of a row containing the value *v*. The intuition here is that to calculate information gain from splitting on a feature we find the weighted average of all potential subsets we might end up with, with the weight being directly proportional to the likelihood of that split occurring. If a dataset contains 99 rows with the value `registered` and only row with the value `unregistered`, we are much more likely to end up with a subset of 99 rows than 1 so we give more weight to the entropy resulting from a value of `registered`.

Our decision tree algorithm works by looking at each column and working out the expecrted information gain resulting from splitting the dataset on that column. At each stage, the column resulting in the largest information gain is chosen. Returning to our census data

In [9]:
df = pd.read_csv('Vegetation.csv')
df = df.set_index('ID') # use the ID column as the index for the dataframe
print(df.columns)

Index(['Stream', 'Slope', 'Elevation', 'Vegetation'], dtype='object')


Assuming now that *Vegetation* is our target variable, we have a choice of 3 columns on which to split the dataset. Let's look at each of these in turn and calculate the information gain resulting from each split. 

In [10]:
current_entropy = calculate_entropy(df, 'Vegetation')
print(f"Current entropy is {current_entropy}")

# split the dataset on stream
print(df['Stream'].unique())

entropy_stream_false = calculate_entropy(df.loc[df['Stream'] == False], 'Vegetation')
entropy_stream_true = calculate_entropy(df.loc[df['Stream'] == True], 'Vegetation')

entropy_stream = (calculate_probability(df, 'Stream', False) * entropy_stream_false) + \
                (calculate_probability(df, 'Stream', True) * entropy_stream_true)

ig_stream = current_entropy - entropy_stream

print(f"Information Gain for stream = {ig_stream}")

# split the dataset on slope
print(df['Slope'].unique())
entropy_slope_flat = calculate_entropy(df.loc[df['Slope'] == 'flat'], 'Vegetation')
entropy_slope_moderate = calculate_entropy(df.loc[df['Slope'] == 'moderate'], 'Vegetation')
entropy_slope_steep = calculate_entropy(df.loc[df['Slope'] == 'steep'], 'Vegetation')

entropy_slope = (calculate_probability(df, 'Slope', 'flat') * entropy_slope_flat) + \
            (calculate_probability(df, 'Slope', 'moderate') * entropy_slope_moderate) + \
            (calculate_probability(df, 'Slope', 'steep') * entropy_slope_steep)

ig_slope = current_entropy - entropy_slope
print(f"Information Gain for slope = {ig_slope}")

Current entropy is 1.5566567074628228
[False  True]
Information Gain for stream = 0.30595849286804166
['steep' 'moderate' 'flat']
Information Gain for slope = 0.5774062828523452


Apply the same logic as above to calculate the information gain for elevation. Which feature should the decision tree choose to split on?

In [11]:
current_entropy = calculate_entropy(df, 'Vegetation')

print(df['Elevation'].unique())

entropy_elevation_highest = calculate_entropy(df.loc[df['Elevation'] == 'highest'], 'Vegetation')
entropy_elevation_high = calculate_entropy(df.loc[df['Elevation'] == 'high'], 'Vegetation')
entropy_elevation_medium = calculate_entropy(df.loc[df['Elevation'] == 'medium'], 'Vegetation')
entropy_elevation_low = calculate_entropy(df.loc[df['Elevation'] == 'low'], 'Vegetation')

entropy_elevation = (calculate_probability(df, 'Elevation', 'highest') * entropy_elevation_highest) + \
    (calculate_probability(df, 'Elevation', 'high') * entropy_elevation_high) + \
    (calculate_probability(df, 'Elevation', 'medium') * entropy_elevation_medium) + \
    (calculate_probability(df, 'Elevation', 'low') * entropy_elevation_low)

ig_elevation = current_entropy - entropy_elevation

print(f"Information Gain for elevation is {ig_elevation}") # 0.878...

['high' 'low' 'medium' 'highest']
Information Gain for elevation is 0.8773870642966131


That was quite a lot of typing, and most of the work was repetitive. We can make our lives a lot easier by rolling this up into a formula. First, we'll start with the definition. What parameters will we need to calulcate the expected information_gain for a column?

In [12]:
def calculate_info_gain(df: pd.DataFrame, split_column: str, target_column: str):
    pass

Next, we want to break our algorithm down into steps, or pseudo-code. Try this before moving on, look back over the previous cells to help guide you

1. Calculate the current entropy
2. Find all unique values for split_column
3. For each unique value *s*, of split_column:
    1. Filter the dataframe to only rows containing *s*
    2. Calculate the entropy of the filtered dataframe
    3. Multiply the entropy by the `|s|/|df|` (the probabiliy of selecting s at random)
4. Sum the outputs for each value of 3.

We're going to start by creating a function to handle A. B.and C. above. Fill in the function below. This function should calculate the probability of selecting split_value randomly from split_column. It should then calculate the entropy on the dataset (filtered by split_value) and return the weighted entropy (probability * entropy)

In [13]:
def calculate_weighted_entropy(df: pd.DataFrame, split_column: str, split_value: str, target_column: str):
    prob = calculate_probability(df, split_column, split_value)
    entropy = calculate_entropy(df.loc[df[split_column] == split_value], target_column)
    weighted = prob * entropy
    return weighted

Now we can carry out steps 1., 2., 3. and 4. Implement the calculate_info_gain function below. You can use the `calculate_weighted_entropy` function above to help you

In [14]:
def calculate_info_gain(df: pd.DataFrame, split_column: str, target_column: str):
    current_entropy = calculate_entropy(df, 'Vegetation')
    return current_entropy - sum([calculate_weighted_entropy(df, split_column, v, target_column) for v in df[split_column].unique()])

print(calculate_info_gain(df, 'Stream', 'Vegetation')) # 0.306
print(calculate_info_gain(df, 'Slope', 'Vegetation')) # 0.578
print(calculate_info_gain(df, 'Elevation', 'Vegetation')) # 0.878

0.30595849286804166
0.5774062828523452
0.8773870642966131


## Choosing the Split Column

We now know how to calculate the expected information gain for any column. All we need to do is find the column with the largest expected information gain, and use that to split the dataset.

In [15]:
def choose_next_column(df: pd.DataFrame, target_column: str):
    # make sure we don't split on the target_column, that's cheating 
    # (and not very useful for an unseen instance)!
    all_columns = [c for c in df.columns.values if c != target_column]
    
    # find the index of the column with the largest info_gain
    chosen_index = np.argmax([calculate_info_gain(df, c, target_column) for c in all_columns])
    
    # return the column with the largest info gain
    return all_columns[chosen_index]


choose_next_column(df, 'Vegetation')

'Elevation'

# Exercises

1. So far we've built a decision tree algorithm which uses Information Gain to determine the next column to split on. Create a new function choose_next_column_igr to use the Info Gain Ratio instead, you may re-use the functions already created in this notebook

2. Create a new function, choose_next_column_gini() which uses the Gini coefficien to determine the next column to split on

3. Load up the census.csv dataset, which column would a decision tree using Info Gain choose to split on first?